[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/01-gpt-4o-research-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/01-gpt-4o-research-agent.ipynb)

# GPT-4o Research Agent in LangGraph

In [1]:
!apt-get install graphviz libgraphviz-dev pkg-config

'apt-get' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Now we install Python libraries:

In [2]:
# Installing required Python libraries
%pip install -qU \
    langchain-pinecone==0.1.1 \
    langchain-openai==0.1.9 \
    langchain==0.2.5 \
    langchain-core==0.2.9 \
    langgraph \
    semantic-router==0.0.48 \
    serpapi==0.1.5 \
    google-search-results==2.4.2 \
    pygraphviz==1.12


Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-312\pygraphviz
      creating build\lib.win-amd64-cpython-312\pygraphviz\tests
      copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-312\pygraphviz\tests
      copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64-cpython-312\pygraphviz\tests
      copying pygraphviz\test

In [3]:
%pip install --upgrade semantic-router
%pip install --upgrade cohere

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U langsmith

  Using cached langsmith-0.3.2-py3-none-any.whl.metadata (14 kB)
Using cached langsmith-0.3.2-py3-none-any.whl (333 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas 0.1.9 requires appdirs, which is not installed.
langchain-community 0.3.16 requires langchain<0.4.0,>=0.3.16, but you have langchain 0.2.5 which is incompatible.
langchain-community 0.3.16 requires langchain-core<0.4.0,>=0.3.32, but you have langchain-core 0.2.9 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 0.2.9 which is incompatible.
langchain 0.2.5 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.3.2 which is incompatible.
langchain-core 0.2.9 requires langsmith<0.2.0,>=0.1.75, but you have langsmith 0.3.2 which is incompatible.
langchain-groq 0.2.3 requires langchain-core<0.4.0,>=0.3.29, but you have langchain-core 0.2.9 which is incompatible.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have 

In [5]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load path from the environment variable
env_ih1 = os.getenv("ENV_IH1")

dotenv_path = Path(env_ih1)
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY= os.getenv('PINECONE_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
STEAMSHIP_API_KEY = os.getenv('STEAMSHIP_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
GEMINI_KEY = os.getenv('GEMINI_KEY')

os.environ['PATH'] += os.pathsep + '/usr/bin'

In [6]:
import openai
from langsmith import wrappers, traceable

LANGSMITH_API_KEY= LANGSMITH_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="memory-project"

### Connect to Pinecone DB

In [7]:

import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("OpenAI API key: ")

# from dotenv import load_dotenv, find_dotenv
import os
# _ = load_dotenv(find_dotenv())

OPENAI_API_KEY = OPENAI_API_KEY
PINECONE_API_KEY = PINECONE_API_KEY

encoder = OpenAIEncoder(
    name="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY #Get the API key from environment variable
)

In [8]:
from pinecone import Pinecone
# Initialize Pinecone
pinecone = Pinecone(
    api_key=PINECONE_API_KEY,  # Replace with your Pinecone API key
    environment="us-east-1"  # Replace with your Pinecone environment (e.g., "us-west1-gcp")
)

In [9]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
# PINECONE_API_KEY = userdata.get("PINECONE_KEY") or "YOUR_API_KEY"

# configure client
pc = Pinecone(api_key=PINECONE_API_KEY)

In [10]:
import time

# Connect to the existing index
index_name = "memory-project"  # Replace with your existing index name
# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [11]:
# from pinecone import ServerlessSpec

# spec = ServerlessSpec(
#     cloud="aws", region="us-west-2"  # us-east-1
# )

In [12]:
dims = len(encoder(["some random text"])[0])
dims

1536

## Graph State

We will define a custom graph state to support our agent-oriented decision making.

In [13]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

There are four parts to our agent state, those are:

* `input`: this is the user's most recent query, usually this would be a question that we want to answer with our research agent.
* `chat_history`: we are building a conversational agent that can support multiple interactions, to allow previous interactions to provide additional context throughout our agent logic we include the chat history in the agent state.
* `intermediate_steps`: provides a record of all steps the research agent will take between the user asking a question via `input` and the agent providing a final answer. This can include things like "search arxiv", "perform general purpose web search", etc. These intermediate steps are crucial to allowing the agent to follow a path of coherent actions and ultimately producing an informed final answer.

## Custom Tools

In [14]:
from langchain_core.tools import tool

ImportError: cannot import name 'ToolOutputMixin' from 'langchain_core.messages.tool' (c:\Users\aurel\OneDrive\Documents\Python\IronHack\W8\D2\lab-langsmith-summarizations\.conda\Lib\site-packages\langchain_core\messages\tool.py)

Let's test the tool.

### Web Search

The web search tool will provide the agent with access to web search. It will be instructed to use this for more general knowledge queries.

In [15]:
from serpapi import GoogleSearch

serpapi_params = {
    "engine": "google",
    "api_key": SERPAPI_API_KEY
}

search = GoogleSearch({
    **serpapi_params,
    "q": "coffee"
})

results = search.get_dict()["organic_results"]

In [16]:
contexts = "\n---\n".join(
    ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
)

In [17]:
print(contexts)

Coffee
Coffee is a beverage brewed from roasted, ground coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, ...
https://en.wikipedia.org/wiki/Coffee
---
Arabica Coffee Prices Hit New High on U.S., Colombia ...
Continuous arabica coffee futures on the ICE were up 0.1% at $3.48 a pound in volatile evening trading in Europe, though they are up more than ...
https://www.wsj.com/economy/trade/arabica-coffee-prices-hit-new-high-on-u-s-colombia-tariff-spat-582446fe
---
r/Coffee
thread where you can share what you are brewing or ask for bean recommendations. This is a place to share and talk about your favorite coffee roasters or beans.
https://www.reddit.com/r/Coffee/
---
Starbucks Coffee Company
More than just great coffee. Explore the menu, sign up for Starbucks® Rewards, manage your gift card and more.
https://www.starbucks.com/
---
Best Rated Coffee Online - Whole Bean & Ground Coffee
Don't look anywhere else to find the best-rated coffee 

We put this process into a tool:

In [18]:
# Install the missing package
%pip install langchain_core

from langchain_core.tools import tool

@tool("web_search")
def web_search(query: str):
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    search = GoogleSearch({
        **serpapi_params,
        "q": query,
        "num": 5
    })
    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
        ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
    )
    return contexts

ImportError: cannot import name 'ToolOutputMixin' from 'langchain_core.messages.tool' (c:\Users\aurel\OneDrive\Documents\Python\IronHack\W8\D2\lab-langsmith-summarizations\.conda\Lib\site-packages\langchain_core\messages\tool.py)

: 

### RAG Tools

We provide two RAG-focused tools for our agent. The `rag_search` allows the agent to perform a simple RAG search for some information across _all_ indexed research papers. The `rag_search_filter` also searches, but _within_ a specific paper which is filtered for via the `arxiv_id` parameter.

We also define the `format_rag_contexts` function to handle the transformation of our Pinecone results from a JSON object to a readble plaintext format.

In [209]:
from langchain_core.tools import tool


def format_rag_contexts(matches: list):
    contexts = []
    for x in matches:
        text = (
            f"Doc name: {x['metadata']['Doc name']}\n"
            f"Author: {x['metadata']['Author']}\n"
            f"Summary: {x['metadata']['Summary']}\n"
            f"Chunks: {x['metadata']['Chunks']}\n"
        )
        contexts.append(text)
    context_str = "\n---\n".join(contexts)
    return context_str



@tool("rag_search")
def rag_search(query: str):
    """Finds related information using a natural language query in the family history."""
    xq = encoder([query])
    xc = index.query(vector=xq, top_k=5, include_metadata=True)
    context_str = format_rag_contexts(xc["matches"])
    return context_str

@tool("rag_search_filter")
def rag_search_filter(query: str, author: str):
    """Finds related information using a natural language query in the family history given an author."""
    xq = encoder([query])
    xc = index.query(vector=xq, top_k=3, include_metadata=True, filter={"Author": author})
    context_str = format_rag_contexts(xc["matches"])
    return context_str

@tool("rag_search")
def rag_search(query: str):
    """Finds specialist information on AI using a natural language query."""
    xq = encoder([query])
    xc = index.query(vector=xq, top_k=2, include_metadata=True)
    context_str = format_rag_contexts(xc["matches"])
    return context_str

In [210]:
rag_search.run("Who is Dreyfus?")

'Doc name: Pdf img\nAuthor: John Doe\nSummary: Isaac, souvent appelé Louis, est l\'ancêtre direct né en 1766 à Froeningen, marié à Rachel Gugenheim (1772-1843). Ils ont eu six enfants, dont Lehmann (Clément), né en 1800, qui a épousé Athelle Bloch en 1827. Ils ont eu neuf enfants, parmi lesquels Jacques Dreyfus, né le 9 janvier 1829, qui a épousé Catherine Lévy en 1860 et est décédé en 1893. Jacques, colporteur de tissus, a joué un rôle significatif dans la famille, voyageant pour approvisionner des revendeurs dans des bourgs non desservis.\nChunks: 118.0\n\n---\nDoc name: La belle au bois dormant\nAuthor: Perrault\nSummary: In "La Belle au Bois Dormant," a princess pricks her finger on a spindle, falling into a deep sleep as foretold by the fairies. Despite efforts to revive her, she remains unconscious. The king places her in a beautifully adorned chamber, where she appears angelic, still breathing but unresponsive. He decides to let her sleep until the time of her awakening. Meanwhi

### Final Answer

Finally, we define a "final answer" tool. This isn't a tool in the usual sense, instead we use it to force a particular output format from our LLM via the function/tool calling.

In [212]:
@tool("final_answer")
def final_answer(
    introduction: str,
    research_steps: str,
    main_body: str,
    conclusion: str,
    sources: Union[str, List[str], None] = None
):
    """
    You are the family safe, keeper of the family collective memory.

    Returns a natural language response to the user iquestion based on the family memory database.

    Do not invent anything but you are allowed to link information together from multiple sources.

    If you can't find an answer with {"tool": "rag_search"}, just say that you don't know.
    """
    if type(research_steps) is list:
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if type(sources) is list:
        sources = "\n".join([f"- {s}" for s in sources])
    return ""

In [297]:
from langchain_core.messages import SystemMessage, AIMessage

@tool("final_answer")
def final_answer(
    introduction: str,
    research_steps: str,
    main_body: str,
    conclusion: str,
    sources: Union[str, List[str], None] = None
):
    """
    You are the family safe, keeper of the family collective memory.

    Returns a natural language response to the user's question based on the family memory database.

    Do not invent anything but you are allowed to link information together from multiple sources.

    If you can't find an answer with sources from {"tool": "rag_search"}, just say that you don't know.
    """

    # Format the research steps and sources
    if isinstance(research_steps, list):
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if isinstance(sources, list):
        sources = "\n".join([f"- {s}" for s in sources])

    # System prompt to guide the final response
    system_prompt = SystemMessage(
        content=(
            "You are the family safe, the guardian of collective memory. "
            "Provide a detailed but concise summary based on the following sections: "
            "introduction, research steps, main body, and conclusion. "
            "Include sources if available and only use verified information."
        )
    )

    # Combine the sections into a message
    user_query_context = (
        f"### Introduction:\n{introduction}\n\n"
        f"### Research Steps:\n{research_steps}\n\n"
        f"### Main Body:\n{main_body}\n\n"
        f"### Conclusion:\n{conclusion}\n\n"
        f"### Sources:\n{sources if sources else 'No sources available.'}"
    )

    # Append the system prompt to messages and call the LLM
    messages = [system_prompt, AIMessage(content=user_query_context)]
    final_response = llm.invoke(messages)  # Call the LLM for a response

    return final_response


## Initialize the "Oracle"

The **Oracle** LLM is our graph's decision maker. It decides which path we should take down our graph. It functions similarly to an agent but is much simpler and reliable.

The Oracle consists of an LLM provided with a set of potential function calls (ie our tools) that it can decide to use — we force it to use _at least_ one of those tool using the `tool_choice="any"` setting (see below). Our Oracle only makes the decision to use a tool, it doesn't execute the tool code itself (we do that seperately in our graph).

### Oracle Prompt

Our prompt for the Oracle will emphasize it's decision making ability within the `system_prompt`, leave a placeholder for us to later insert `chat_history`, and provide a place for us to insert the user `input`.

In [298]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """You are the Family Safe, keeper of the family's collective memory. 
Your role is to decide how to handle user queries using the available tools.

**Tool Usage:**
- Do NOT reuse a tool for the same query (check the scratchpad).
- Do NOT use any tool more than **twice**.
- Prioritize **rag_search** for gathering information.
- Use **web_search** only for additional historical context.

**Response Protocol:**
- If **rag_search** provides no answer, state: "I don't know."
- NEVER invent information or use data beyond the family memory.
- Always provide sources via the **final_answer** tool.

By following these rules, you ensure accurate and responsible responses."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("assistant", "scratchpad: {scratchpad}"),
])

Next, we must initialize our `llm` (for this we use `gpt-4o`) and then create the _runnable_ pipeline of our Oracle.

The runnable connects our inputs (the user `input` and `chat_history`) to our `prompt`, and our `prompt` to our `llm`. It is also where we _bind_ our tools to the LLM and enforce function calling via `tool_choice="any"`.

In [299]:
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key = OPENAI_API_KEY,
    temperature=0
)

tools=[
    rag_search_filter,
    rag_search,
    # web_search,
    final_answer
]

# define a function to transform intermediate_steps from list
# of AgentAction to scratchpad string
def create_scratchpad(intermediate_steps: list[AgentAction]):
    research_steps = []
    for i, action in enumerate(intermediate_steps):
        if action.log != "TBD":
            # this was the ToolExecution
            research_steps.append(
                f"Tool: {action.tool}, input: {action.tool_input}\n"
                f"Output: {action.log}"
            )
    return "\n---\n".join(research_steps)

oracle = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "scratchpad": lambda x: create_scratchpad(
            intermediate_steps=x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

Test the agent quickly to confirm it is functional:

In [300]:
inputs = {
    "input": "tell me something interesting about the Dreyfus family",
    "chat_history": [],
    "intermediate_steps": [],
}
out = oracle.invoke(inputs)
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qpczlLaIXRz6805hsr6Yylh6', 'function': {'arguments': '{"query":"interesting facts","author":"Dreyfus family"}', 'name': 'rag_search_filter'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 358, 'total_tokens': 383, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4c94be86-9884-4b75-bbb3-1b75507f53d6-0', tool_calls=[{'name': 'rag_search_filter', 'args': {'query': 'interesting facts', 'author': 'Dreyfus family'}, 'id': 'call_qpczlLaIXRz6805hsr6Yylh6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 358, 'output_tokens': 25, 'total_tokens': 383, 'input_toke

It is running but we are returning a lot of output here, we can narrow this down to what we need — ie, the chosen tool name and generated input args for the tool.

In [301]:
out.tool_calls[0]["name"]

'rag_search_filter'

In [302]:
out.tool_calls[0]["args"]

{'query': 'interesting facts', 'author': 'Dreyfus family'}

We can see now that our Oracle decided to use the `web_search` tool with a `query` of `"interesting facts about dogs"` — a good choice.

## Define Nodes for Graph

We will be passing the tool use decision to our `router` which will _route_ the output to the chosen node component to run (we define these below) based on the `out.tool_calls[0]["name"]` value.

In [303]:
def run_oracle(state: list):
    print("run_oracle")
    print(f"intermediate_steps: {state['intermediate_steps']}")
    out = oracle.invoke(state)
    tool_name = out.tool_calls[0]["name"]
    tool_args = out.tool_calls[0]["args"]
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log="TBD"
    )
    return {
        "intermediate_steps": [action_out]
    }

def router (state: list): #Original
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
#         return "final_answer"
    
# def router(state: list):
#     if isinstance(state[-1].get("intermediate_steps"), list):  # Access the last state entry
#         # Return the tool name from the last intermediate step
#         return state[-1]["intermediate_steps"][-1]["tool"]
#     else:
#         # Handle invalid format or empty intermediate steps
#         print("Router: Invalid or missing intermediate steps. Routing to final_answer.")
#         return "final_answer"  

All of our tools can be run using the same function logic, which we define with `run_tool`. The input parameters to our tool call and the resultant output are added to our graph state's `intermediate_steps` parameter.

In [304]:
tool_str_to_func = {
    "rag_search_filter": rag_search_filter,
    "rag_search": rag_search,
    # "fetch_arxiv": fetch_arxiv,
    # "web_search": web_search,
    "final_answer": final_answer
}

def run_tool(state: list):
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool
    tool_args = state["intermediate_steps"][-1].tool_input
    print(f"{tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name].invoke(input=tool_args)
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )
    return {"intermediate_steps": [action_out]}

## Define Graph

In [305]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("oracle", run_oracle)
graph.add_node("rag_search_filter", run_tool)
graph.add_node("rag_search", run_tool)
# graph.add_node("fetch_arxiv", run_tool)
# graph.add_node("web_search", run_tool)
graph.add_node("final_answer", run_tool)

graph.set_entry_point("oracle")

graph.add_conditional_edges(
    source="oracle",  # where in graph to start
    path=router,  # function to determine which node is called
)

# create edges from each tool back to the oracle
for tool_obj in tools:
    if tool_obj.name != "final_answer":
        graph.add_edge(tool_obj.name, "oracle")

# if anything goes to final answer, it must then move to END
graph.add_edge("final_answer", END)

runnable = graph.compile()

In [312]:
# Install the missing package
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [313]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


display(Image(runnable.get_graph().draw_mermaid_png()))

ModuleNotFoundError: No module named 'langchain_core.runnables.graph_mermaid'

In [ ]:
# Install pygraphviz
%pip install pygraphviz==1.12 

from IPython.display import Image

Image(runnable.get_graph().draw_png())


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-312\pygraphviz
      copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-312\pygraphviz
      creating build\lib.win-amd64-cpython-312\pygraphviz\tests
      copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-312\pygraphviz\tests
      copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64-cpython-312\pygraphviz\tests
      copying pygraphviz\test

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

## Building Reports

Let's test our research agent. First, I want to try on something simple (although not within the intended use-case of our agent):

In [306]:
out = runnable.invoke({
    "input": "Who is Dreyfus?",
    "chat_history": [],
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'Dreyfus'})
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'Dreyfus'}, log='TBD'), AgentAction(tool='rag_search', tool_input={'query': 'Dreyfus'}, log='Doc name: Pdf img\nAuthor: John Doe\nSummary: Isaac, souvent appelé Louis, est l\'ancêtre direct né en 1766 à Froeningen, marié à Rachel Gugenheim (1772-1843). Ils ont eu six enfants, dont Lehmann (Clément), né en 1800, qui a épousé Athelle Bloch en 1827. Ils ont eu neuf enfants, parmi lesquels Jacques Dreyfus, né le 9 janvier 1829, qui a épousé Catherine Lévy en 1860 et est décédé en 1893. Jacques, colporteur de tissus, a joué un rôle significatif dans la famille, voyageant pour approvisionner des revendeurs dans des bourgs non desservis.\nChunks: 118.0\n\n---\nDoc name: La belle au bois dormant\nAuthor: Perrault\nSummary: In "La Belle au Bois Dormant," a princess pricks her finger on a spindle, falling into a deep sleep as foretold

Let's create a function to consume the agent output and format it into our report:

In [280]:
def build_report(output: dict):
    research_steps = output["research_steps"]
    if type(research_steps) is list:
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    sources = output["sources"]
    if type(sources) is list:
        sources = "\n".join([f"- {s}" for s in sources])
    return f"""
INTRODUCTION
------------
{output["introduction"]}

RESEARCH STEPS
--------------
{research_steps}

REPORT
------
{output["main_body"]}

CONCLUSION
----------
{output["conclusion"]}

SOURCES
-------
{sources}
"""

In [281]:
print(build_report(
    output=out["intermediate_steps"][-1].tool_input
))


INTRODUCTION
------------
The name Dreyfus is associated with a significant figure in family history.

RESEARCH STEPS
--------------
I conducted a search for information on Dreyfus and found relevant details about Jacques Dreyfus, an ancestor in the family lineage.

REPORT
------
Jacques Dreyfus was born on January 9, 1829, and married Catherine Lévy in 1860. He was a cloth peddler who played an important role in the family by traveling to supply retailers in underserved towns. His father was Isaac Dreyfus, often referred to as Louis, who was born in 1766 and married Rachel Gugenheim. The Dreyfus family has a rich history with several notable members.

CONCLUSION
----------
Jacques Dreyfus's contributions to the family and his role as a peddler highlight the family's entrepreneurial spirit during that era.

SOURCES
-------
- rag_search



Now let's try with an on-topic question on AI.

In [294]:
out = runnable.invoke({
    "input": "Do we have info about La Belle au bois dormant in the family database?",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search_filter.invoke(input={'query': 'La Belle au bois dormant', 'author': 'family'})
run_oracle
intermediate_steps: [AgentAction(tool='rag_search_filter', tool_input={'query': 'La Belle au bois dormant', 'author': 'family'}, log='TBD'), AgentAction(tool='rag_search_filter', tool_input={'query': 'La Belle au bois dormant', 'author': 'family'}, log='')]
rag_search_filter.invoke(input={'query': 'La Belle au bois dormant', 'author': 'family'})
run_oracle
intermediate_steps: [AgentAction(tool='rag_search_filter', tool_input={'query': 'La Belle au bois dormant', 'author': 'family'}, log='TBD'), AgentAction(tool='rag_search_filter', tool_input={'query': 'La Belle au bois dormant', 'author': 'family'}, log=''), AgentAction(tool='rag_search_filter', tool_input={'query': 'La Belle au bois dormant', 'author': 'family'}, log='TBD'), AgentAction(tool='rag_search_filter', tool_input={'query': 'La Belle au bois dormant', 'author': 'family'}, log='')]
rag_search_

In [295]:
print(build_report(
    output=out["intermediate_steps"][-1].tool_input
))


INTRODUCTION
------------
The query regarding 'La Belle au bois dormant' has been explored.

RESEARCH STEPS
--------------
I searched the family database for any information related to 'La Belle au bois dormant' but found no relevant entries.

REPORT
------
Unfortunately, there is no information available in the family database regarding 'La Belle au bois dormant.'

CONCLUSION
----------
If you have any specific aspects or details you would like to know about, please let me know, and I can assist further.

SOURCES
-------
None



Let's ask about RAG specifically.

In [286]:
out = runnable.invoke({
    "input": "What's the history of the Dreyfus?",
    "chat_history": []
})

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'Dreyfus history'})
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'Dreyfus history'}, log='TBD'), AgentAction(tool='rag_search', tool_input={'query': 'Dreyfus history'}, log='Doc name: Pdf img\nAuthor: John Doe\nSummary: Isaac, souvent appelé Louis, est l\'ancêtre direct né en 1766 à Froeningen, marié à Rachel Gugenheim (1772-1843). Ils ont eu six enfants, dont Lehmann (Clément), né en 1800, qui a épousé Athelle Bloch en 1827. Ils ont eu neuf enfants, parmi lesquels Jacques Dreyfus, né le 9 janvier 1829, qui a épousé Catherine Lévy en 1860 et est décédé en 1893. Jacques, colporteur de tissus, a joué un rôle significatif dans la famille, voyageant pour approvisionner des revendeurs dans des bourgs non desservis.\nChunks: 118.0\n\n---\nDoc name: La belle au bois dormant\nAuthor: Perrault\nSummary: In "La Belle au Bois Dormant," a princess pricks her finger on a spindle, falling into 

In [287]:
print(build_report(
    output=out["intermediate_steps"][-1].tool_input
))


INTRODUCTION
------------
The Dreyfus family has a notable history, particularly through its ancestor Jacques Dreyfus.

RESEARCH STEPS
--------------
I found information about the Dreyfus family lineage, starting with Isaac, often called Louis, who was born in 1766. He married Rachel Gugenheim and had six children. One of his descendants, Jacques Dreyfus, born in 1829, played a significant role in the family's history.

REPORT
------
Jacques Dreyfus married Catherine Lévy in 1860 and passed away in 1893. He was a traveling fabric merchant, which was significant for the family's economic activities, as he supplied goods to retailers in underserved areas. This aspect of his life highlights the family's involvement in trade and commerce during that period.

CONCLUSION
----------
The Dreyfus family's history reflects a lineage of merchants and their contributions to local economies through trade.

SOURCES
-------
{'query': 'Dreyfus history', 'author': 'John Doe'}



---